# LoRA

### LoRA를 활용한 GPT-2 감성 분석 모델 튜닝

In [60]:
!pip install peft datasets transformers hf_transfer

In [42]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

In [ ]:
# 환경 변수 설정 (local에서 진행할 때는 token 설정 필요)
# from huggingface_hub import login
# login(token="hf_XXX") # 실제 HF_TOKEN 입력

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [44]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [45]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### 1. 기반 모델 load

In [46]:
model_name = 'gpt2'

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
base_model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### 2. LoRA 설정 (LoraConfig)

In [47]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=['c_attn', 'c_fc', 'c_proj'],
    lora_dropout=0.1,
    task_type='SEQ_CLS',
    bias='none'
)

In [48]:
# 기반 모델 + LoRA 설정 == 학습할 모델
model = get_peft_model(base_model, lora_config).to(device)

### 3. 데이터 준비

In [49]:
dataset = load_dataset('imdb')

In [50]:
# 학습용 데이터 준비
# 샘플링 편향 방지
pos_sample = [data for data in dataset['train'] if data['label'] == 1][:500]
neg_sample = [data for data in dataset['train'] if data['label'] == 0][:500]

# X, y 데이터 구분
train_texts = [data['text'] for data in pos_sample + neg_sample]
train_labels = [data['label'] for data in pos_sample + neg_sample]

In [51]:
# 검증용 데이터 준비
# 샘플링 편향 방지
pos_eval = [data for data in dataset['test'] if data['label'] == 1][:500]
neg_eval = [data for data in dataset['test'] if data['label'] == 0][:500]

# X, y 데이터 구분
eval_texts = [data['text'] for data in pos_eval + neg_eval]
eval_labels = [data['label'] for data in pos_eval + neg_eval]

In [52]:
def preprocess_data(texts, labels):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )
    encodings['labels'] = torch.tensor(labels, dtype=torch.long)

    return encodings

In [53]:
train_encodings = preprocess_data(train_texts, train_labels)
eval_encodings = preprocess_data(eval_texts, eval_labels)

In [54]:
class IMDBDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        return {
            key: val[idx] for key, val in self.encodings.items()
        }

In [55]:
train_dataset = IMDBDataset(train_encodings)
eval_dataset = IMDBDataset(eval_encodings)

In [56]:
# 데이터 묶음 처리(batch 처리하는 함수)
def collate_fn(batch):
    batch = {
        key: torch.stack([item[key] for item in batch]) for key in batch[0]
    }
    return batch

### 4. 학습 준비

In [57]:
# 학습 파라미터 설정(학습에 대한 설정)
training_args = TrainingArguments(
    output_dir='./results',                      # 결과 저장 경로
    per_device_train_batch_size=4,               # 학습 배치 크기
    per_device_eval_batch_size=4,                # 검증 배치 크기
    num_train_epochs=5,                          # 학습 반복 횟수
    save_steps=100,                              # 저장 주기
    save_total_limit=2,                          # 최대 2개의 모델 저장
    eval_strategy='epoch',                       # 검증 주기
    logging_dir='./logs',                        # 로그 저장 경로
    logging_steps=10,                            # 로그 출력 주기
    fp16=torch.cuda.is_available()               # GPU 있으면 fp16, 없으면 자동으로 False
)

print(f"🔧 학습 설정 완료!")
print(f"   - Device: {device}")
print(f"   - FP16: {training_args.fp16}")
print(f"   - Epochs: {training_args.num_train_epochs}")
print(f"   - Batch size: {training_args.per_device_train_batch_size}")

🔧 학습 설정 완료!
   - Device: cpu
   - FP16: False
   - Epochs: 5
   - Batch size: 4


In [58]:
# Trainer 설정 (학습할 모델 + 학습 파라미터(학습 설정) + 데이터셋 + 데이터 처리 함수 + 데이터 묶음 처리 함수)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn
)

In [ ]:
# local 에서 학습 진행되는 것을 확인하고, runpod 에서 실제 학습 진행
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

### 5. 추론

In [64]:
def predict_sentiment(text):
    inputs = tokenizer(
        text,
         return_tensors='pt',
         padding=True,
         truncation=True,
         max_length=512
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()

        return "긍정" if prediction == 1 else "부정"


In [65]:
test_review = "I love this movie!"
result = predict_sentiment(test_review)
result

'부정'

In [66]:
# base model 추론 결과와 비교

reload_base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

def predict_sentiment_origin(text):
    inputs = tokenizer(
        text,
         return_tensors='pt',
         padding=True,
         truncation=True,
         max_length=512
    ).to(device)
    
    with torch.no_grad():
        outputs = reload_base_model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=-1).item()

        return "긍정" if prediction == 1 else "부정"


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
test_review = "I love this movie!"
result = predict_sentiment_origin(test_review)
result

'긍정'